In [1]:
import argparse
import os
from tqdm.notebook import tqdm
import numpy as np
import mindspore as ms
from mindspore import ops
from mindnlp.transformers import (
    BertGenerationTokenizer,
    BertGenerationDecoder,
    BertGenerationConfig,
    CLIPModel,
    CLIPTokenizer
)
from loaders.ZO_Clip_loaders import cifar100_single_isolated_class_loader
from sklearn.metrics import roc_auc_score
from mindspore import context
import sys

[WARNING] GE_ADPT(65263,ffffb1820b80,python):2025-01-16-01:39:24.401.591 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleGetModelId failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleGetModelId
[WARNING] GE_ADPT(65263,ffffb1820b80,python):2025-01-16-01:39:24.401.644 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleLoadFromMem failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleLoadFromMem
[WARNING] GE_ADPT(65263,ffffb1820b80,python):2025-01-16-01:39:24.401.668 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleUnload failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleUnload
[WARNING] GE_ADPT(65263,ffffb1820b80,python):2025-01-16-01:39:24.401.801 [mindspore/ccsrc/utils/dlopen_macro.h:163] Dl

In [2]:
def tokenize_for_clip(batch_sentences, tokenizer):
    # 使用CLIPTokenizer直接处理
    inputs = tokenizer(
        batch_sentences,
        padding=True,
        truncation=True,
        max_length=77,
        return_tensors="ms"
    )
    return inputs.input_ids

In [3]:
def greedysearch_generation_topk(clip_embed, bert_model, batch_size=32):
    N = clip_embed.shape[0]
    max_len = 77

    # 初始化batch的target序列
    target_lists = [[berttokenizer.bos_token_id] for _ in range(N)]
    top_k_lists = [[] for _ in range(N)]
    bert_model.set_train(False)

    for i in range(max_len):
        targets = ms.Tensor(target_lists, dtype=ms.int64)
        position_ids = ms.Tensor(np.arange(targets.shape[1])[None].repeat(N, axis=0), ms.int32)
        attention_mask = ops.ones((N, targets.shape[1]), dtype=ms.int32)

        out = bert_model(
            input_ids=targets,
            attention_mask=attention_mask,
            position_ids=position_ids,
            encoder_hidden_states=clip_embed,
        )

        pred_idxs = out.logits.argmax(axis=2)[:, -1].astype(ms.int64)
        _, top_k = ops.topk(out.logits, dim=2, k=35)

        for j in range(N):
            target_lists[j].append(pred_idxs[j].item())
            top_k_lists[j].append(top_k[j, -1])

        if all(len(t) >= 10 for t in target_lists):
            break

    results = []
    for i in range(N):
        top_k_tensor = ops.concat(top_k_lists[i])
        target_tensor = ms.Tensor(target_lists[i], dtype=ms.int64)
        results.append((target_tensor, top_k_tensor))

    return results


In [4]:
def image_decoder(clip_model, berttokenizer, image_loaders=None, bert_model=None):
    splits = [list(range(20)), list(range(20, 40)), list(range(40, 60)),
              list(range(60, 80)), list(range(80, 100))]
    auc_list_sum = []

    for seen_split in splits[-1::-1]:
        split = seen_split + list(set(list(range(100))) - set(seen_split))
        label_split = [idx2class[idx] for idx in split]
        seen_labels = label_split[:20]
        seen_descriptions = [f"This is a photo of a {label}" for label in seen_labels]
        targets = ms.Tensor(2000 * [0] + 8000 * [1], dtype=ms.int32)
        max_num_entities = 0
        ood_probs_sum = []
        print(seen_labels)

        for semantic_label in tqdm(label_split):
            loader = image_loaders[semantic_label]

            for batch_data in loader.create_dict_iterator():
                batch_images = batch_data["image"]
                batch_size = batch_images.shape[0]

                clip_model.set_train(False)
                clip_out = clip_model.get_image_features(pixel_values=batch_images)
                clip_extended_embed = ops.repeat_elements(clip_out, rep=2, axis=1)
                clip_extended_embed = ops.expand_dims(clip_extended_embed, 1)

                batch_results = greedysearch_generation_topk(clip_extended_embed, bert_model)
                del clip_extended_embed
                del clip_out

                batch_target_tokens = []
                batch_topk_tokens = []

                for target_list, topk_list in batch_results:
                    target_tokens = [berttokenizer.decode(int(pred_idx.asnumpy())) for pred_idx in target_list]
                    topk_tokens = [berttokenizer.decode(int(pred_idx.asnumpy())) for pred_idx in topk_list]
                    batch_target_tokens.append(target_tokens)
                    batch_topk_tokens.append(topk_tokens)

                batch_unique_entities = []
                for topk_tokens in batch_topk_tokens:
                    unique_entities = list(set(topk_tokens) - set(seen_labels))
                    batch_unique_entities.append(unique_entities)
                    max_num_entities = max(max_num_entities, len(unique_entities))

                batch_all_desc = []
                for unique_entities in batch_unique_entities:
                    all_desc = seen_descriptions + [f"This is a photo of a {label}" for label in unique_entities]
                    batch_all_desc.append(all_desc)

                batch_all_desc_ids = [tokenize_for_clip(all_desc, cliptokenizer) for all_desc in batch_all_desc]

                image_features = clip_model.get_image_features(pixel_values=batch_images)
                image_features = image_features / ops.norm(image_features, dim=-1, keepdim=True)

                for b_idx in range(len(batch_results)):
                    text_features = clip_model.get_text_features(input_ids=batch_all_desc_ids[b_idx])
                    text_features = text_features / ops.norm(text_features, dim=-1, keepdim=True)

                    similarity = 100.0 * (image_features[b_idx:b_idx + 1] @ text_features.T)
                    zeroshot_probs = ops.softmax(similarity, axis=-1).squeeze()

                    ood_prob_sum = float(ops.sum(zeroshot_probs[20:]).asnumpy())
                    ood_probs_sum.append(ood_prob_sum)

                del batch_target_tokens
                del batch_topk_tokens
                del batch_unique_entities
                del batch_all_desc
                del image_features

        auc_sum = roc_auc_score(targets.asnumpy(), np.array(ood_probs_sum))
        print('当前split的sum_ood AUROC={}'.format(auc_sum))
        auc_list_sum.append(auc_sum)

    print('所有AUC分数:', auc_list_sum)
    print('AUC均值和标准差:', np.mean(auc_list_sum), np.std(auc_list_sum))


In [5]:
def get_args_in_notebook():
    args = argparse.Namespace(
        trained_path='./trained_models/COCO/'
    )
    return args

In [6]:
class_to_idx= {
            'apple': 0,
            'aquarium_fish': 1,
            'baby': 2,
            'bear': 3,
            'beaver': 4,
            'bed': 5,
            'bee': 6,
            'beetle': 7,
            'bicycle': 8,
            'bottle': 9,
            'bowl': 10,
            'boy': 11,
            'bridge': 12,
            'bus': 13,
            'butterfly': 14,
            'camel': 15,
            'can': 16,
            'castle': 17,
            'caterpillar': 18,
            'cattle': 19,
            'chair': 20,
            'chimpanzee': 21,
            'clock': 22,
            'cloud': 23,
            'cockroach': 24,
            'couch': 25,
            'crab': 26,
            'crocodile': 27,
            'cup': 28,
            'dinosaur': 29,
            'dolphin': 30,
            'elephant': 31,
            'flatfish': 32,
            'forest': 33,
            'fox': 34,
            'girl': 35,
            'hamster': 36,
            'house': 37,
            'kangaroo': 38,
            'keyboard': 39,
            'lamp': 40,
            'lawn_mower': 41,
            'leopard': 42,
            'lion': 43,
            'lizard': 44,
            'lobster': 45,
            'man': 46,
            'maple_tree': 47,
            'motorcycle': 48,
            'mountain': 49,
            'mouse': 50,
            'mushroom': 51,
            'oak_tree': 52,
            'orange': 53,
            'orchid': 54,
            'otter': 55,
            'palm_tree': 56,
            'pear': 57,
            'pickup_truck': 58,
            'pine_tree': 59,
            'plain': 60,
            'plate': 61,
            'poppy': 62,
            'porcupine': 63,
            'possum': 64,
            'rabbit': 65,
            'raccoon': 66,
            'ray': 67,
            'road': 68,
            'rocket': 69,
            'rose': 70,
            'sea': 71,
            'seal': 72,
            'shark': 73,
            'shrew': 74,
            'skunk': 75,
            'skyscraper': 76,
            'snail': 77,
            'snake': 78,
            'spider': 79,
            'squirrel': 80,
            'streetcar': 81,
            'sunflower': 82,
            'sweet_pepper': 83,
            'table': 84,
            'tank': 85,
            'telephone': 86,
            'television': 87,
            'tiger': 88,
            'tractor': 89,
            'train': 90,
            'trout': 91,
            'tulip': 92,
            'turtle': 93,
            'wardrobe': 94,
            'whale': 95,
            'willow_tree': 96,
            'wolf': 97,
            'woman': 98,
            'worm': 99
        }

In [7]:
if __name__ == '__main__':
    # 判断是否在notebook环境
    if 'ipykernel' in sys.modules or 'IPython' in sys.modules:
        args = get_args_in_notebook()
        context.set_context(device_target="Ascend")
    else:
        parser = argparse.ArgumentParser()
        parser.add_argument('--trained_path', type=str, default='./trained_models/COCO/')
        args = parser.parse_args()
        context.set_context(device_target="Ascend")

    args.saved_model_path = args.trained_path + '/ViT-B32/'

    if not os.path.exists(args.saved_model_path):
        os.makedirs(args.saved_model_path)

    # 初始化tokenizers
    berttokenizer = BertGenerationTokenizer.from_pretrained('google/bert_for_seq_generation_L-24_bbc_encoder')

    # 加载CLIP模型和tokenizer
    model_name = 'openai/clip-vit-base-patch32'
    try:
        clip_model = CLIPModel.from_pretrained(model_name)
        cliptokenizer = CLIPTokenizer.from_pretrained(model_name)
    except Exception as e:
        print(f"Error loading model from mirror, trying direct download: {e}")
        clip_model = CLIPModel.from_pretrained(model_name)
        cliptokenizer = CLIPTokenizer.from_pretrained(model_name)

    # 初始化BERT模型
    if (not os.path.exists(f"{args.saved_model_path}/decoder_model")):
        bert_config = BertGenerationConfig.from_pretrained("google/bert_for_seq_generation_L-24_bbc_encoder")
        bert_config.is_decoder = True
        bert_config.add_cross_attention = True
        bert_config.return_dict = True
        bert_model = BertGenerationDecoder.from_pretrained("google/bert_for_seq_generation_L-24_bbc_encoder",
                                                           config=bert_config)
    else:
        bert_model = BertGenerationDecoder.from_pretrained(f"{args.saved_model_path}/decoder_model")
    print('t0')
    cifar100_loaders = cifar100_single_isolated_class_loader()
    print('t1')
    # 获取CIFAR100的类别映射
    idx2class = {v: k for k, v in class_to_idx.items()}

    image_decoder(clip_model, berttokenizer, image_loaders=cifar100_loaders, bert_model=bert_model)

[WARNING] ME(65263:281473659833216,MainProcess):2025-01-16-01:39:36.218.227 [mindspore/run_check/_check_version.py:329] MindSpore version 2.4.10 and Ascend AI software package (Ascend Data Center Solution)version 7.3 does not match, the version of software package expect one of ['7.5', '7.6']. Please refer to the match info on: https://www.mindspore.cn/install
[WARNING] ME(65263:281473659833216,MainProcess):2025-01-16-01:39:36.219.982 [mindspore/run_check/_check_version.py:407] Can not find the tbe operator implementation(need by mindspore-ascend). Please check whether the Environment Variable PYTHONPATH is set. For details, refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(65263:281473659833216,MainProcess):2025-01-16-01:39:36.220.634 [mindspore/run_check/_check_version.py:347] MindSpore version 2.4.10 and "te" wheel package version 7.3 does not match. For details, refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(652

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(
BertGenerationDecoder has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


t0


  0%|          | 0/100 [00:00<?, ?it/s]

...

当前split的sum_ood AUROC=0.83000275
['chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard']


  0%|          | 0/100 [00:00<?, ?it/s]

当前split的sum_ood AUROC=0.8249155000000001
['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle']


  0%|          | 0/100 [00:00<?, ?it/s]

当前split的sum_ood AUROC=0.81786125
所有AUC分数: [0.8057389375, 0.8656406249999999, 0.83000275, 0.8249155000000001, 0.81786125]
AUC均值和标准差: 0.8288318125 0.020125761566801608
